# Setup

To run the example, a file named `secret.py` must be created in the notebooks
folder with the following content:

```python
# Configuration settings

# SLIPO workbench installation
BASE_URL = 'https://app.dev.slipo.eu'

# SLIPO API key
API_KEY = ''
```

The `API_KEY` value must be set to a valid SLIPO Application Key. The file must be imported before creating a new context:

```python
from secret import BASE_URL, API_KEY
```

# Requirements

The following modules must be installed:

- descartes
- geopandas
- urbanaccess
- pandana
- slipo-loci

For Ubuntu 18.04, python3-rtree package must be installed e.g.

`sudo apt install python3-rtree`

In [ ]:
# Create new context
from slipoframes.context import SlipoContext

from secret import BASE_URL, API_KEY

ctx = SlipoContext(
    base_url = BASE_URL,
    requires_ssl = False,
    api_key = API_KEY
)

# Data loading

In this section, we will query the SLIPO service for the most recent workflow executions. Then we will copy the output file of the last API call from the example `Demo_02_Workflow`.

In [ ]:
ctx.process_query(pageIndex=0, pageSize=5)

In [ ]:
# Replace process id value from the most recent export operation
export1 = ctx.process_status(461, 1)

In [ ]:
export1.output()

In [ ]:
ctx.process_file_download(export1.output(), target='./output/exported-data.zip', overwrite=True)

# Data Processing

Once data have been saved locally, analysis can be perfomed using
tools like pandas DataFrames, geopanadas GeoDataFrames or other libraries 
like https://github.com/paulhoule/gastrodon for RDF.

In [ ]:
# Unzip output file
import os
import zipfile

with zipfile.ZipFile('./output/exported-data.zip','r') as zip_ref:
    zip_ref.extractall("./output/")
    
os.rename('./output/points.csv', './output/Fuel_Berlin.csv')

In [ ]:
# Load CSV data in a DataFrame
import pandas as pd

pois = pd.read_csv('./output/Fuel_Berlin.csv', delimiter='|', error_bad_lines=False)

pois.head()

In [ ]:
# Since shapely does not supports Extended Well-Known Text, we update the geometry and remove the SRID value
pois['the_geom'] = pois['the_geom'].apply(lambda x: x.split(';')[1])
pois.head()

In [ ]:
# Create a GeoDataFrame
import geopandas
from shapely import wkt

pois['the_geom'] = pois['the_geom'].apply(wkt.loads)

gdf = geopandas.GeoDataFrame(pois, geometry='the_geom')

print(gdf.head())

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))

# We restrict to Romania.
ax = world[world.name == 'Germany'].plot(
    color='white', edgecolor='black')

# We can now plot our GeoDataFrame.
gdf.plot(ax=ax, color='blue')

plt.show()

In [ ]:
import loci as lc
from loci import io
from loci import analytics
from loci import plots

In [ ]:
pois = lc.io.read_poi_csv(input_file='./output/Fuel_Berlin.csv',
                       col_id='id',
                       col_name='name',
                       col_lon='lon',
                       col_lat='lat',
                       col_kwds='category',
                       col_sep='|',
                       kwds_sep=',',
                       source_crs='EPSG:4326',
                       target_crs='EPSG:4326',
                       keep_other_cols=False)
pois.head(10)

In [ ]:
# Rename category column
pois.rename(columns={'category': 'kwds'}, inplace=True)

In [ ]:
m = lc.plots.map_points(pois.sample(100), show_bbox=True)
m

In [ ]:
kf = lc.analytics.kwds_freq(pois)
kf

In [ ]:
lc.plots.barchart(kf, plot_title='Top Keywords', x_axis_label='Keywords', y_axis_label='Frequency')

In [ ]:
lc.plots.plot_wordcloud(pois)

In [ ]:
pois_filtered = lc.analytics.filter_by_kwd(pois, 'FUEL')
lc.plots.heatmap(pois_filtered, radius=12)